# 식품의약품안전처_묶음의약품정보서비스 (식의약헬스종합정보)
---
- 작성자 : 안지은
- 실행일 : 2023-01-10

In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

In [2]:
# 총 데이터 수 구하는 함수
def get_total_cnt(key):
    url = 'https://apis.data.go.kr/1471000/DrbBundleInfoService02/getDrbBundleList02?serviceKey=%s&type=json&cnsgnItemName='%(key)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['body']['totalCount'])
    return cnt

In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def get_api_conts(key,page):
    
    url = 'https://apis.data.go.kr/1471000/DrbBundleInfoService02/getDrbBundleList02?serviceKey=%s&pageNo=%s&numOfRows=100&type=json&cnsgnItemName='%(key,str(page))
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['body']['items']
    
    return data

### API 가져오기
---
- 한 페이지 당  100개 씩 제한
- 총 51454 개 행


In [4]:
# 발매 채널별
drug_bundle = []
cnt = get_total_cnt(key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts(key,i)
    for item in js:
        drug_bundle.append(item)


100%|██████████| 515/515 [07:54<00:00,  1.09it/s]


### 데이터 프레임 변환
---
- 컬럼명 : API 제공 페이지 기준

In [5]:
drug_bundle_df = pd.json_normalize(drug_bundle)
drug_bundle_df.columns = ['대표 품목기준코드','대표 제품명', '대표 주성분', '대표 함량', '대표 업소명','대표 제조소',
'대표 허가일자','대표 심평원주성분코드','대표 심평원제품코드','대표 ATC코드','취소/취하명','품목기준코드','제품명',
'업소명','제조소','허가일자','심평원제품코드','취소/취하명']
drug_bundle_df

,대표 품목기준코드,대표 제품명,대표 주성분,대표 함량,대표 업소명,대표 제조소,대표 허가일자,대표 심평원주성분코드,대표 심평원제품코드,대표 ATC코드,취소/취하명,품목기준코드,제품명,업소명,제조소,허가일자,심평원제품코드,취소/취하명
0,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
1,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
2,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
3,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
4,196900031,대한에피네프린주사액,에피네프린,1밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-10-23 00:00:00,152630BIJ,645100741,C01CA24 (epinephrine),정상,202103993,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-20 00:00:00,None,정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51449,200403232,레보니아정(레보노르게스트렐),레보노르게스트렐,0.75밀리그램,명문제약(주),명문제약(주) 화성 제 1공장,2004-10-11 00:00:00,None,None,G03AD01 (levonorgestrel),유효기간만료,200600232,엠에스필정(레보노르게스트렐),태극제약(주),명문제약(주) 화성 제 1공장,2006-02-09 00:00:00,None,유효기간만료
51450,200403232,레보니아정(레보노르게스트렐),레보노르게스트렐,0.75밀리그램,명문제약(주),명문제약(주) 화성 제 1공장,2004-10-11 00:00:00,None,None,G03AD01 (levonorgestrel),유효기간만료,200600011,레보노민정(레보노르게스트렐),신풍제약(주),명문제약(주) 화성 제 1공장,2006-02-03 00:00:00,None,유효기간만료
51451,199900565,소아용록시틸과립(록시트로마이신),록시트로마이신,50밀리그램,코오롱제약(주),코오롱제약(주) 대전 제 1공장,1999-02-18 00:00:00,None,None,J01FA06 (roxithromycin),취하,200606692,아이록스과립(록시트로마이신),한올바이오파마(주),코오롱제약(주) 대전 제 1공장,2006-11-15 00:00:00,None,취하
51452,199900565,소아용록시틸과립(록시트로마이신),록시트로마이신,50밀리그램,코오롱제약(주),코오롱제약(주) 대전 제 1공장,1999-02-18 00:00:00,None,None,J01FA06 (roxithromycin),취하,200606692,아이록스과립(록시트로마이신),한올바이오파마(주),코오롱제약(주) 대전 제 1공장,2006-11-15 00:00:00,None,취하
